In [3]:
from tensorflow.keras.models import Sequential #모델링
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer #토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences #패딩

In [4]:
texts = ['자연어 처리 알고리즘', '자연어 처리 방법', '자연어 NLP 알고리즘 알고리즘', '자연어 처리 전문가']

In [5]:
tok = Tokenizer()
tok.fit_on_texts(texts)

In [6]:
tok.index_word #각 단어에 대한 인덱스 부여

{1: '자연어', 2: '처리', 3: '알고리즘', 4: '방법', 5: 'nlp', 6: '전문가'}

In [7]:
#text를 matrix로 변환해준다. - 현재 각 문장에 들어가있는 단어들 출력 (default 값이 binary인 것)
tok.texts_to_matrix(texts) #첫번째 0은 아무런 의미가 존재하지 않는 것

array([[0., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 1.]])

In [10]:
tok.texts_to_matrix(texts, mode = 'binary') #단어의 유/무

array([[0., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 1.]])

In [11]:
tok.texts_to_matrix(texts, mode = 'count')  #각 문장에 있는 단어의 빈도수를 나타낸다.

array([[0., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 0., 1., 0., 0.],
       [0., 1., 0., 2., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 1.]])

In [12]:
#TF-IDF (단어의 빈도수 - 단어가 나타난 문서의 개수) - 흔하지 않게 나타날 수록 중요도가 올라간다.
tok.texts_to_matrix(texts, mode = 'tfidf')

array([[0.        , 0.58778666, 0.69314718, 0.84729786, 0.        ,
        0.        , 0.        ],
       [0.        , 0.58778666, 0.69314718, 0.        , 1.09861229,
        0.        , 0.        ],
       [0.        , 0.58778666, 0.        , 1.43459998, 0.        ,
        1.09861229, 0.        ],
       [0.        , 0.58778666, 0.69314718, 0.        , 0.        ,
        0.        , 1.09861229]])

In [13]:
# 각 문장을 기준으로 단어의 빈도를 비율로 나타낸다.
tok.texts_to_matrix(texts, mode = 'freq')

array([[0.        , 0.33333333, 0.33333333, 0.33333333, 0.        ,
        0.        , 0.        ],
       [0.        , 0.33333333, 0.33333333, 0.        , 0.33333333,
        0.        , 0.        ],
       [0.        , 0.25      , 0.        , 0.5       , 0.        ,
        0.25      , 0.        ],
       [0.        , 0.33333333, 0.33333333, 0.        , 0.        ,
        0.        , 0.33333333]])

In [14]:
import pandas as pd
from string import punctuation #특수기호
import numpy as np

In [20]:
df = pd.read_csv('NYT_2018.csv') #headline이 news title
df

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,5ae82c93068401528a2ab969,1004,By CLAIRE CAIN MILLER,article,This Common Question Reinforces the Gender Pay...,"['Discrimination', 'Wages and Salaries', 'Labo...",68,Upshot,3,2018-05-01 09:00:01,Unknown,Several states and cities have ordered employe...,The New York Times,News,https://www.nytimes.com/2018/05/01/upshot/how-...
1320,5ae82c95068401528a2ab96b,1043,By TRACY J. GATES,article,"Anna, Llama and Me","['Friendship', 'Dewdney, Anna', 'Writing and W...",65,Well,0,2018-05-01 09:00:02,Family,"The beginning, middle and end of a picture boo...",The New York Times,News,https://www.nytimes.com/2018/05/01/well/family...
1321,5ae82c9d068401528a2ab96d,659,Interview by AUDIE CORNISH,article,Gen. Michael Hayden Has One Regret: Russia,"['Classified Information and State Secrets', '...",66,Magazine,70,2018-05-01 09:00:06,Unknown,"The former N.S.A. and C.I.A. chief on Trump, S...",The New York Times,News,https://www.nytimes.com/2018/05/01/magazine/ge...
1322,5ae82c9f068401528a2ab96f,1155,By JASON ROBERT BROWN,article,There Is Nothin’ Like a Tune,"['Books and Literature', 'Purdum, Todd S', 'Th...",68,BookReview,17,2018-05-01 09:00:07,Book Review,"In “Something Wonderful,” Todd S. Purdum analy...",The New York Times,Review,https://www.nytimes.com/2018/05/01/books/revie...


In [21]:
df.headline #series로 data 형태가 추출된다.

0       Former N.F.L. Cheerleaders’ Settlement Offer: ...
1       E.P.A. to Unveil a New Rule. Its Effect: Less ...
2                                 The New Noma, Explained
3                                                 Unknown
4                                                 Unknown
                              ...                        
1319    This Common Question Reinforces the Gender Pay...
1320                                   Anna, Llama and Me
1321           Gen. Michael Hayden Has One Regret: Russia
1322                         There Is Nothin’ Like a Tune
1323                                              Unknown
Name: headline, Length: 1324, dtype: object

In [22]:
df.headline.values #배열 형태로 추출

array(['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
       'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
       'The New Noma, Explained', ...,
       'Gen. Michael Hayden Has One Regret: Russia',
       'There Is Nothin’ Like a Tune', 'Unknown'], dtype=object)

In [23]:
type(df.headline.values)

numpy.ndarray

In [24]:
#리스트로 변환
headline = []
headline.extend(list(df.headline.values)) #headline에 있는 값들을 list로 변환해준다.
headline

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Unknown',
 'Unknown',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.',
 'In Berlin, artists find a home',
 'Unknown',
 'The Right Stuff',
 'Jimmy Carter Knows What North Korea Wants',
 'The Truth Is Out There',
 'New Jersey Ruling Could Reignite Battle Over Church-State Separation',
 'Procrastinating',
 'Word + Quiz: dilatory',
 'My Life-Threatening Bout With E. Coli Food Poisoning',
 'Choosing Brexit, a Town Yearned for Its Seafa

In [25]:
df.isnull().sum() #null 값 (결측치 존재 하지 않는다.)

articleID           0
articleWordCount    0
byline              0
documentType        0
headline            0
keywords            0
multimedia          0
newDesk             0
printPage           0
pubDate             0
sectionName         0
snippet             0
source              0
typeOfMaterial      0
webURL              0
dtype: int64

In [26]:
#headline이 'Unkown'이라고 설정된 요소들 삭제하기
len(headline)

1324

In [27]:
# list에서 headline이 'Unknown' 요소의 개수 출력하기
char_headline = 'Unknown'
count = 0

for line in headline:
  if line == char_headline:
    count += 1
  else:
    continue

count

110

In [30]:
#데이터프레임에서 headaline이 Unknown인 값들 찾기 - 이렇게 찾는 것이 훨씬 시간 단축 가능
sum(df.headline == 'Unknown')

110

In [31]:
#Unknown 값들을 제거해서 리스트에 갱신하기
headline = [w for w in headline if w != 'Unknown']
len(headline)

1214

### headline에 저장된 뉴스 기사 제목으로 다음 단어를 생성하는 LSTM 기반 모델 설계하기

**동작 예시**

입력 : I (생성하고자 하는 단어의 개수)

출력 : I was . . .

In [32]:
#데이터프레임 전처리 작업

headline[0].lower() # 특수문자, 따옴표, 등 존재하는 것을 알 수 있다.

'former n.f.l. cheerleaders’ settlement offer: $1 and a meeting with goodell'

In [33]:
headline[1000]

'N0 Exp3r13nc3? N0 pr0bl3m!'

In [34]:
punctuation #특수문자 모음집

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [35]:
def pre_func(title):
  #따옴표 빼고 다 제거되었음 .. 이게 글자 하나씩 받아오는 거라서 join으로 묶어준다.
  res = ''.join(w.lower() for w in title if w not in punctuation)
  return res

In [36]:
pre_headline = [pre_func(x) for x in headline]
len(pre_headline)

1214

In [37]:
#단어로 Vocabulary 만들어주기
tok = Tokenizer()

In [38]:
tok.fit_on_texts(pre_headline) #pre_headline이 리스트라서 대괄호 안쳐줌

In [39]:
type(tok.index_word)

dict

In [40]:
tok.index_word

{1: 'the',
 2: 'a',
 3: 'to',
 4: 'of',
 5: 'in',
 6: 'for',
 7: 'and',
 8: 'on',
 9: 'is',
 10: 'with',
 11: 'trump',
 12: 'as',
 13: 'at',
 14: 'new',
 15: 'from',
 16: 'how',
 17: 'it',
 18: 'an',
 19: 'be',
 20: 'that',
 21: 'season',
 22: 'us',
 23: 'you',
 24: 'what',
 25: 'episode',
 26: 'can',
 27: 'your',
 28: 'he',
 29: 'not',
 30: 'his',
 31: 'are',
 32: 'teaching',
 33: 'out',
 34: 'was',
 35: 'no',
 36: 'by',
 37: 'trump’s',
 38: 'has',
 39: 'war',
 40: 'over',
 41: 'may',
 42: 'into',
 43: 'more',
 44: 'now',
 45: 'why',
 46: 'who',
 47: 'about',
 48: 'recap',
 49: 'activities',
 50: '1',
 51: 'we',
 52: 'do',
 53: 'women',
 54: 'when',
 55: 'syria',
 56: 'trade',
 57: 'it’s',
 58: '2',
 59: 'or',
 60: 'will',
 61: 'i',
 62: 'have',
 63: 'just',
 64: 'but',
 65: 'home',
 66: 'up',
 67: 'this',
 68: 'one',
 69: 'off',
 70: 'house',
 71: 'president',
 72: 'gop',
 73: 'our',
 74: 'long',
 75: 'facebook',
 76: 'case',
 77: 'life',
 78: 'end',
 79: 'some',
 80: 'power',
 81: '

In [41]:
len(tok.index_word)

3619

In [42]:
tok.word_index.items() #튜플로 출력

dict_items([('the', 1), ('a', 2), ('to', 3), ('of', 4), ('in', 5), ('for', 6), ('and', 7), ('on', 8), ('is', 9), ('with', 10), ('trump', 11), ('as', 12), ('at', 13), ('new', 14), ('from', 15), ('how', 16), ('it', 17), ('an', 18), ('be', 19), ('that', 20), ('season', 21), ('us', 22), ('you', 23), ('what', 24), ('episode', 25), ('can', 26), ('your', 27), ('he', 28), ('not', 29), ('his', 30), ('are', 31), ('teaching', 32), ('out', 33), ('was', 34), ('no', 35), ('by', 36), ('trump’s', 37), ('has', 38), ('war', 39), ('over', 40), ('may', 41), ('into', 42), ('more', 43), ('now', 44), ('why', 45), ('who', 46), ('about', 47), ('recap', 48), ('activities', 49), ('1', 50), ('we', 51), ('do', 52), ('women', 53), ('when', 54), ('syria', 55), ('trade', 56), ('it’s', 57), ('2', 58), ('or', 59), ('will', 60), ('i', 61), ('have', 62), ('just', 63), ('but', 64), ('home', 65), ('up', 66), ('this', 67), ('one', 68), ('off', 69), ('house', 70), ('president', 71), ('gop', 72), ('our', 73), ('long', 74), ('

In [43]:
idx2word = {} #dictionary
for k, v in tok.word_index.items(): #튜플에 값들을 딕셔너리로 넣어준다.
  idx2word[v] = k #index = key, vocabulary = value (인덱스를 통해 단어를 찾아줄 것이기 때문이다.)

In [44]:
idx2word #빈도 수대로 자동으로 나열 (근데 tok.index_word 이거랑 똑같은 결과가 나온다 ...  굳이 왜 해줌?)

{1: 'the',
 2: 'a',
 3: 'to',
 4: 'of',
 5: 'in',
 6: 'for',
 7: 'and',
 8: 'on',
 9: 'is',
 10: 'with',
 11: 'trump',
 12: 'as',
 13: 'at',
 14: 'new',
 15: 'from',
 16: 'how',
 17: 'it',
 18: 'an',
 19: 'be',
 20: 'that',
 21: 'season',
 22: 'us',
 23: 'you',
 24: 'what',
 25: 'episode',
 26: 'can',
 27: 'your',
 28: 'he',
 29: 'not',
 30: 'his',
 31: 'are',
 32: 'teaching',
 33: 'out',
 34: 'was',
 35: 'no',
 36: 'by',
 37: 'trump’s',
 38: 'has',
 39: 'war',
 40: 'over',
 41: 'may',
 42: 'into',
 43: 'more',
 44: 'now',
 45: 'why',
 46: 'who',
 47: 'about',
 48: 'recap',
 49: 'activities',
 50: '1',
 51: 'we',
 52: 'do',
 53: 'women',
 54: 'when',
 55: 'syria',
 56: 'trade',
 57: 'it’s',
 58: '2',
 59: 'or',
 60: 'will',
 61: 'i',
 62: 'have',
 63: 'just',
 64: 'but',
 65: 'home',
 66: 'up',
 67: 'this',
 68: 'one',
 69: 'off',
 70: 'house',
 71: 'president',
 72: 'gop',
 73: 'our',
 74: 'long',
 75: 'facebook',
 76: 'case',
 77: 'life',
 78: 'end',
 79: 'some',
 80: 'power',
 81: '

In [45]:
vocab_size = len(tok.index_word) + 1 #0번 숫자 때문에 1을 더해준 것 (관행) - Padding 토큰(짧은 길이)를 위한 추가적인 인덱스 확보

In [28]:
df.headline

0       Former N.F.L. Cheerleaders’ Settlement Offer: ...
1       E.P.A. to Unveil a New Rule. Its Effect: Less ...
2                                 The New Noma, Explained
3                                                 Unknown
4                                                 Unknown
                              ...                        
1319    This Common Question Reinforces the Gender Pay...
1320                                   Anna, Llama and Me
1321           Gen. Michael Hayden Has One Regret: Russia
1322                         There Is Nothin’ Like a Tune
1323                                              Unknown
Name: headline, Length: 1324, dtype: object

In [46]:
#Encoding
#LSTM Model로 다음 단어를 예측하기 위해서는 아래와 같이 설계해야한다.
sequences = []
for s in pre_headline: #제목에 있는 것들
  enc = tok.texts_to_sequences([s])[0] # 각 문장 별 인코딩 결과 - 대괄호를 통해 리스트로 만든다. (문장에 들어있는 단어 인덱스 출력)
  print(enc)
  for i in range(1, len(enc)): # 각 문장 단위로 읽어진다.
    seq = enc[:i+1] # 그 다음에 예측해야할 단어들 (이 다음엔 이게 나올거야! 이런 느낌) - 준비작업
    sequences.append(seq)

[95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10, 1102]
[96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367, 5, 1105]
[1, 14, 1106, 1107]
[16, 2, 574, 4, 210, 368, 1108, 2, 369, 575]
[9, 162, 2, 130, 6, 1109]
[1110, 1111]
[1112, 1113, 163, 370, 6, 2, 1114, 57, 86, 370]
[1115, 576, 3, 371, 13, 577, 1116, 64, 578, 579, 211, 1117]
[1118, 1119]
[28, 580, 1, 1120, 3, 1121, 212, 213, 44, 211, 365, 10, 372, 1122]
[5, 1123, 581, 214, 2, 65]
[1, 87, 373]
[264, 582, 1124, 24, 88, 164, 374]
[1, 215, 9, 33, 216]
[14, 583, 375, 131, 1125, 97, 40, 1126, 1127]
[1128]
[265, 584, 1129]
[98, 585, 1130, 10, 1131, 1132, 586, 1133]
[587, 1134, 2, 376, 1135, 6, 85, 1136, 132, 7, 1137, 85, 588]
[2, 1138, 1139]
[377, 373, 266, 378]
[13, 1, 165, 1, 112, 267, 41, 19, 589]
[133, 4, 1140, 379, 64, 28, 268, 380, 134]
[60, 1, 166, 381, 66, 3, 99, 11]
[269, 5, 217]
[16, 590, 135, 1141, 576, 8, 382, 1142]
[18, 218, 383, 5, 1143]
[51, 89, 90, 35, 1144]
[591, 1145, 7, 219, 9, 1146, 36, 592]
[270, 220]
[74, 271, 221, 1147, 113, 79

In [ ]:
sequences[:10] # 2단어 ~ (마지막 숫자)까지 리스트를 생성함.

[[95, 263],
 [95, 263, 1100],
 [95, 263, 1100, 1101],
 [95, 263, 1100, 1101, 572],
 [95, 263, 1100, 1101, 572, 50],
 [95, 263, 1100, 1101, 572, 50, 7],
 [95, 263, 1100, 1101, 572, 50, 7, 2],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10, 1102]]

**[슬라이싱 기법을 사용하는 이유]**

문장 가정: "LSTM 모델로 단어를 예측하도록 설계합니다."

1번.

입력: LSTM

출력: 모델로

</br>
2번.

입력: LSTM 모델로

출력: 단어를

...

</br>
N번.

입력: LSTM 모델로 단어를 예측하도록

출력: 설계합니다.

</br>

Return Sequences : False (예측값 1개 출력 - N:1 구조)

Retrun Sequences : True (예측값 여러 개 출력 - N:N 구조)

위에 상황은 (N:N 구조) LSTM 모델을 2번 생성했을 때 위의 모델이 아래 모델로 출력 값을 전달해줄 때 사용한다.

</br>
오차를 줄이기 위해 뒤에서 앞으로 이동한다. (오차역전파)

In [47]:
sequences

[[95, 263],
 [95, 263, 1100],
 [95, 263, 1100, 1101],
 [95, 263, 1100, 1101, 572],
 [95, 263, 1100, 1101, 572, 50],
 [95, 263, 1100, 1101, 572, 50, 7],
 [95, 263, 1100, 1101, 572, 50, 7, 2],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10, 1102],
 [96, 3],
 [96, 3, 1103],
 [96, 3, 1103, 2],
 [96, 3, 1103, 2, 14],
 [96, 3, 1103, 2, 14, 573],
 [96, 3, 1103, 2, 14, 573, 85],
 [96, 3, 1103, 2, 14, 573, 85, 1104],
 [96, 3, 1103, 2, 14, 573, 85, 1104, 366],
 [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367],
 [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367, 5],
 [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367, 5, 1105],
 [1, 14],
 [1, 14, 1106],
 [1, 14, 1106, 1107],
 [16, 2],
 [16, 2, 574],
 [16, 2, 574, 4],
 [16, 2, 574, 4, 210],
 [16, 2, 574, 4, 210, 368],
 [16, 2, 574, 4, 210, 368, 1108],
 [16, 2, 574, 4, 210, 368, 1108, 2],
 [16, 2, 574, 4, 210, 368, 1108, 2, 369],
 [16, 2, 574, 4, 210, 368, 1108, 2, 369, 57

In [48]:
ml = max(len(i) for i in sequences) #24개의 단어를 기준으로 padding을 한다.

In [50]:
sequences[0] #배열 길이 24개

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,  95, 263], dtype=int32)

In [52]:
#문장의 길이를 전부 다 24로 만들어준다. (왜? LSTM 알고리즘은 무조건 리스트의 길이가 동일해야한다.)
sequences = pad_sequences(sequences, maxlen = ml, padding = 'pre')

In [53]:
sequences[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,  95, 263], dtype=int32)

In [54]:
sequences.shape

(7809, 24)

In [55]:
np.array(sequences)

array([[   0,    0,    0, ...,    0,   95,  263],
       [   0,    0,    0, ...,   95,  263, 1100],
       [   0,    0,    0, ...,  263, 1100, 1101],
       ...,
       [   0,    0,    0, ...,    9, 3619,  110],
       [   0,    0,    0, ..., 3619,  110,    2],
       [   0,    0,    0, ...,  110,    2, 1014]], dtype=int32)

In [56]:
#sequences에 최소 숫자 2개가 나온다 (당연히 단어 2개부터 시작하니까 ...)
#입력값과 출력값을 나누는 작업을 해야한다.
#LSTM에 들어가는 값들의 type은 array이다. (list를 array로 변환 필요 - numpy 사용)

# X = []
# y = []

# for elem in sequences:
#     X.append(elem[:ml-1])
#     y.append(elem[-1])

X = sequences[:,:-1] # [행 (:) - 전체 다, 열 (:-1) -1번째 값 빼고 전체 다 가져온다]
y = sequences[:,-1]  #[행 (:) - 전체 다, 열 -1번째 값만 가져온다]

In [ ]:
X.shape

(7809, 23)

In [ ]:
y.shape

(7809,)

In [ ]:
#훈련, 테스트 데이터 만들
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False, random_state= 0)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, vocab_size) #출력층을 3260개로 만들어준다. (가장 높은 확률의 단어를 추출해낸다.)
y_test = to_categorical(y_test, vocab_size) #테스트 데이터도 마찬가지로 설정한다.

In [ ]:
X_train.shape

(7028, 23)

In [ ]:
y_train.shape

(7028, 3620)

In [ ]:
X_test.shape

(781, 23)

In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
#y에 대해서 One-Hot-Encoding 필요 (확률이기 때문에 1차원은 Cost 측정 불가)
#단어의 크기만큼 array가 나와야한다. (3620개)
#단어가 3,620개이므로, 3,620차원으로 만들어줘야 한다. (크기가 너무 크기 때문에 Embedding 하자!)
model = Sequential()
model.add(Embedding(vocab_size, 50)) #거의 0은 없고 1이기 때문에 차원 축소를 해줘도 되는 것
#LSTM Cell에서 출력값을 하이퍼파라미터로 설정 (숫자의 의미? : 연산의 결과로써 특성값으로 나타낸다. = 모든 LSTM이 128차원으로 출력)
model.add(LSTM(128, return_sequences = True))  #아래쪽에 있는 LSTM이 위에 있는 LSTM에 정보 전달
model.add(LSTM(64, return_sequences = False))
model.add(Dense(vocab_size, activation= 'softmax')) #dense : 1:1 Fully Connected, 입력값: 128, 출력값: 3620

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          181000    
                                                                 
 lstm (LSTM)                 (None, None, 128)         91648     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 3620)              235300    
                                                                 
Total params: 557,356
Trainable params: 557,356
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# from keras.callbacks import EarlyStopping
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',
              metrics = ['accuracy'])
model.fit(X_train, y_train, epochs = 300, verbose = 2, validation_split = 0.2) #epoch로 20만개 문장으로 학습을 한다

Epoch 1/300
176/176 - 21s - loss: 7.7110 - accuracy: 0.0270 - val_loss: 7.6561 - val_accuracy: 0.0356 - 21s/epoch - 122ms/step
Epoch 2/300
176/176 - 3s - loss: 7.1145 - accuracy: 0.0301 - val_loss: 7.8543 - val_accuracy: 0.0199 - 3s/epoch - 16ms/step
Epoch 3/300
176/176 - 2s - loss: 6.9244 - accuracy: 0.0292 - val_loss: 7.9764 - val_accuracy: 0.0441 - 2s/epoch - 14ms/step
Epoch 4/300
176/176 - 2s - loss: 6.7601 - accuracy: 0.0372 - val_loss: 8.1867 - val_accuracy: 0.0405 - 2s/epoch - 13ms/step
Epoch 5/300
176/176 - 2s - loss: 6.6586 - accuracy: 0.0439 - val_loss: 8.3625 - val_accuracy: 0.0448 - 2s/epoch - 11ms/step
Epoch 6/300
176/176 - 2s - loss: 6.5853 - accuracy: 0.0441 - val_loss: 8.3331 - val_accuracy: 0.0405 - 2s/epoch - 12ms/step
Epoch 7/300
176/176 - 2s - loss: 6.5096 - accuracy: 0.0486 - val_loss: 8.5514 - val_accuracy: 0.0427 - 2s/epoch - 9ms/step
Epoch 8/300
176/176 - 2s - loss: 6.4333 - accuracy: 0.0486 - val_loss: 8.7184 - val_accuracy: 0.0370 - 2s/epoch - 12ms/step
Epoch 

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 2) #오버피팅 된듯
print(score[1])

25/25 - 0s - loss: 22.1487 - accuracy: 0.0499 - 153ms/epoch - 6ms/step
0.04993597790598869


입력 - '나는' 10개 예상

나는 - 모델 - '나는 지금' - 모델 - '나는 지금 집에' - 모델 - '나는 지금 집에 있는데' - 모델 - '나는 지금 TV를' -

......

모델 - '나는 지금 집에 있는데 TV를 보고 있다.'

In [ ]:
def gen_sent(model, tok, c_word, n): #c_word에 저장된 단어 뒤의 n가지 문장을 예상을 해주세요
  # 구현
  pred_sent = ''
  for i in range(n):
    enc = tok.texts_to_sequences([c_word])[0] #'The' = 1번 인덱스 (C_WORD를 인덱스로 변환한 값을 나타낸다.)
    print(enc)
    enc = pad_sequences([enc], maxlen = ml-1, padding = 'pre') #2차원 구조로 저장되어져 있는 변수 ([1, 0, 0, ...., 0, 9])
    # 다음 단어를 예측하려면 마지막 예측 단어 제외해서 23개만 넣어야한다.
    # print(enc)
    res = model.predict(enc)
    # print(res.shape)
    #최댓값을 만들어주는 인덱스를 알아내서 dict에서 단어를 뽑아낸다.
    res = np.argmax(res) #가장 최댓값을 가지는 인덱스 뽑아내기
    for w, i in tok.word_index.items():
      if i == res: #인덱스가 가리키는 단어를 추출해낸다.
        break
    #c_word + " " + w #예측한 값 c_word에 넣어줘서 다시 예측하기
    c_word += " " + w #c_word에 추가해주기
    # print("예측 단어: ", w)

  pred_sent = pred_sent + c_word
  return pred_sent

In [ ]:
gen_sent(model, tok, 'The', 15) #랜덤하게 초기 weight가 설정되기 때문에 업데이트하면서 값들이 달라진다.

1/1 [==============================] - 0s 22ms/step


'The new shingles vaccine reasons to get it new life online tied to dementia scientists trump'

#### 과제 1

자연어처리_Day03에서 했던 코드에서 불용어 전처리하기

#### 과제 2

자연어처리_Day04에서 모델 구조를 변경해서 성능 개선하기

Accuracy 증가 여부 확인하기

#### 과제 3 (추가)

우리나라 소설 일부 발췌하여 모델링 연습하기

(토지소설 구글링) - 띄어쓰기로 분리, 한자 제거, 혹은 명사만 뽑